# App processes cleaning

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import gc
import yaml
import json
import requests
import numpy as np
import pandas as pd

from pymongo import MongoClient

from utils import get_files, memory_usage, \
correct_encoding, update_page_metadata

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')
DATA_DIR = os.path.join(BASE_PATH, 'data')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'app_processes'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)
    
client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [5]:
csv_files = get_files(os.path.join(DATA_DIR, model, '*.csv'))

In [6]:
if not csv_files:
    raise FileNotFoundError("Couldn't find any csv files! Please make sure the filepath exists")

In [7]:
df = pd.read_csv(csv_files[0], sep=';', nrows=100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 100 non-null    int64 
 1   sample_id          100 non-null    int64 
 2   name               100 non-null    object
 3   application_label  81 non-null     object
 4   is_system_app      100 non-null    int64 
 5   version_name       81 non-null     object
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


In [8]:
df.describe(include=np.number)

,id,sample_id,is_system_app
count,100.000000,100.000000,100.000000
mean,50.500000,1.620000,0.590000
std,29.011492,0.632136,0.494311
min,1.000000,1.000000,0.000000
25%,25.750000,1.000000,0.000000
50%,50.500000,2.000000,1.000000
75%,75.250000,2.000000,1.000000
max,100.000000,3.000000,1.000000


In [9]:
df.describe(exclude=np.number)

,name,application_label,version_name
count,100,81,81
unique,46,38,29
top,com.kms.free,LG System Server,6.0.1
freq,3,3,16


In [10]:
def save_df_page(page, chunks, collection):
    # concatenate data chunks -> careful benchmark this
    df = pd.concat(chunks, axis=0, ignore_index=True)
    
    collection.update_one({}, {'$inc': {'nrows': df.shape[0]}})

    # store page metadata in mongo
    update_page_metadata(collection, df, exclude=[])

    # save output to a parquet file with brotli compression
    df.to_parquet(os.path.join(model_storage, '{}.{}.parquet'.format(
        model, page)), compression='brotli', index=False)
    
    del df

    return page + 1

In [11]:
mappings = {'sample_id':'uint32', 'domain':'category', 'application_label':'category',
            'is_system_app':'bool'}

model_metadata.update_one({}, {'$set': {'mappings': mappings}})

In [12]:
total_rows = 0
total_memory_usage = 0.0
memory_usage_split = 0.0
last_id = 0

page = 0
chunks = []

features = ['sample_id', 'name', 'application_label', 'is_system_app']

string_columns = df[features].select_dtypes(include='object').columns.to_list()

In [13]:
print('Total csv files: {}'.format(len(csv_files)))

Total csv files: 5912


In [14]:
for filepath in csv_files:
    df = pd.read_csv(filepath, sep=';', usecols=features)

    # drop missing values
    df = df.dropna()

    # basic string pre-processing: convert to lowercase and strip blank chars
    for column in string_columns:
        df.loc[:, column] = [x.lower().strip() for x in df[column].tolist()]

    df['domain'] = [x.split(':', 1)[0] for x in df['name'].values]
    
    df = df.drop(['name'], axis=1)
    
    df = df.astype(mappings)

    total_rows += df.shape[0]
    page_memory_usage = memory_usage(df)
    total_memory_usage += page_memory_usage
    memory_usage_split += page_memory_usage

    chunks.append(df)

    if memory_usage_split >= config['MEMORY_USAGE_SPLIT']:
        print('Page {} created!'.format(page))
        page = save_df_page(page, chunks, model_metadata)

        del chunks[:]
        del chunks
        gc.collect()

        chunks = []
        memory_usage_split = 0.0

    print('{} ✔'.format(filepath))

../../data/app_processes/app_processes.query.1.csv ✔
../../data/app_processes/app_processes.query.2.csv ✔
../../data/app_processes/app_processes.query.3.csv ✔
../../data/app_processes/app_processes.query.4.csv ✔
../../data/app_processes/app_processes.query.5.csv ✔
../../data/app_processes/app_processes.query.6.csv ✔
../../data/app_processes/app_processes.query.7.csv ✔
../../data/app_processes/app_processes.query.8.csv ✔
../../data/app_processes/app_processes.query.9.csv ✔
../../data/app_processes/app_processes.query.10.csv ✔
../../data/app_processes/app_processes.query.11.csv ✔
../../data/app_processes/app_processes.query.12.csv ✔
../../data/app_processes/app_processes.query.13.csv ✔
../../data/app_processes/app_processes.query.14.csv ✔
../../data/app_processes/app_processes.query.15.csv ✔
../../data/app_processes/app_processes.query.16.csv ✔
../../data/app_processes/app_processes.query.17.csv ✔
../../data/app_processes/app_processes.query.18.csv ✔
../../data/app_processes/app_processe

../../data/app_processes/app_processes.query.151.csv ✔
../../data/app_processes/app_processes.query.152.csv ✔
../../data/app_processes/app_processes.query.153.csv ✔
../../data/app_processes/app_processes.query.154.csv ✔
Page 5 created!
../../data/app_processes/app_processes.query.155.csv ✔
../../data/app_processes/app_processes.query.156.csv ✔
../../data/app_processes/app_processes.query.157.csv ✔
../../data/app_processes/app_processes.query.158.csv ✔
../../data/app_processes/app_processes.query.159.csv ✔
../../data/app_processes/app_processes.query.160.csv ✔
../../data/app_processes/app_processes.query.161.csv ✔
../../data/app_processes/app_processes.query.162.csv ✔
../../data/app_processes/app_processes.query.163.csv ✔
../../data/app_processes/app_processes.query.164.csv ✔
../../data/app_processes/app_processes.query.165.csv ✔
../../data/app_processes/app_processes.query.166.csv ✔
../../data/app_processes/app_processes.query.167.csv ✔
../../data/app_processes/app_processes.query.168.

../../data/app_processes/app_processes.query.299.csv ✔
../../data/app_processes/app_processes.query.300.csv ✔
../../data/app_processes/app_processes.query.301.csv ✔
../../data/app_processes/app_processes.query.302.csv ✔
../../data/app_processes/app_processes.query.303.csv ✔
../../data/app_processes/app_processes.query.304.csv ✔
../../data/app_processes/app_processes.query.305.csv ✔
../../data/app_processes/app_processes.query.306.csv ✔
../../data/app_processes/app_processes.query.307.csv ✔
../../data/app_processes/app_processes.query.308.csv ✔
../../data/app_processes/app_processes.query.309.csv ✔
../../data/app_processes/app_processes.query.310.csv ✔
Page 11 created!
../../data/app_processes/app_processes.query.311.csv ✔
../../data/app_processes/app_processes.query.312.csv ✔
../../data/app_processes/app_processes.query.313.csv ✔
../../data/app_processes/app_processes.query.314.csv ✔
../../data/app_processes/app_processes.query.315.csv ✔
../../data/app_processes/app_processes.query.316

../../data/app_processes/app_processes.query.447.csv ✔
../../data/app_processes/app_processes.query.448.csv ✔
../../data/app_processes/app_processes.query.449.csv ✔
../../data/app_processes/app_processes.query.450.csv ✔
../../data/app_processes/app_processes.query.451.csv ✔
../../data/app_processes/app_processes.query.452.csv ✔
../../data/app_processes/app_processes.query.453.csv ✔
../../data/app_processes/app_processes.query.454.csv ✔
../../data/app_processes/app_processes.query.455.csv ✔
../../data/app_processes/app_processes.query.456.csv ✔
../../data/app_processes/app_processes.query.457.csv ✔
../../data/app_processes/app_processes.query.458.csv ✔
../../data/app_processes/app_processes.query.459.csv ✔
../../data/app_processes/app_processes.query.460.csv ✔
../../data/app_processes/app_processes.query.461.csv ✔
../../data/app_processes/app_processes.query.462.csv ✔
../../data/app_processes/app_processes.query.463.csv ✔
../../data/app_processes/app_processes.query.464.csv ✔
../../data

../../data/app_processes/app_processes.query.595.csv ✔
../../data/app_processes/app_processes.query.596.csv ✔
../../data/app_processes/app_processes.query.597.csv ✔
../../data/app_processes/app_processes.query.598.csv ✔
Page 22 created!
../../data/app_processes/app_processes.query.599.csv ✔
../../data/app_processes/app_processes.query.600.csv ✔
../../data/app_processes/app_processes.query.601.csv ✔
../../data/app_processes/app_processes.query.602.csv ✔
../../data/app_processes/app_processes.query.603.csv ✔
../../data/app_processes/app_processes.query.604.csv ✔
../../data/app_processes/app_processes.query.605.csv ✔
../../data/app_processes/app_processes.query.606.csv ✔
../../data/app_processes/app_processes.query.607.csv ✔
../../data/app_processes/app_processes.query.608.csv ✔
../../data/app_processes/app_processes.query.609.csv ✔
../../data/app_processes/app_processes.query.610.csv ✔
../../data/app_processes/app_processes.query.611.csv ✔
../../data/app_processes/app_processes.query.612

...

../../data/app_processes/app_processes.query.5301.csv ✔
../../data/app_processes/app_processes.query.5302.csv ✔
../../data/app_processes/app_processes.query.5303.csv ✔
../../data/app_processes/app_processes.query.5304.csv ✔
../../data/app_processes/app_processes.query.5305.csv ✔
../../data/app_processes/app_processes.query.5306.csv ✔
../../data/app_processes/app_processes.query.5307.csv ✔
../../data/app_processes/app_processes.query.5308.csv ✔
../../data/app_processes/app_processes.query.5309.csv ✔
../../data/app_processes/app_processes.query.5310.csv ✔
../../data/app_processes/app_processes.query.5311.csv ✔
../../data/app_processes/app_processes.query.5312.csv ✔
../../data/app_processes/app_processes.query.5313.csv ✔
../../data/app_processes/app_processes.query.5314.csv ✔
../../data/app_processes/app_processes.query.5315.csv ✔
../../data/app_processes/app_processes.query.5316.csv ✔
../../data/app_processes/app_processes.query.5317.csv ✔
../../data/app_processes/app_processes.query.531

../../data/app_processes/app_processes.query.5446.csv ✔
../../data/app_processes/app_processes.query.5447.csv ✔
../../data/app_processes/app_processes.query.5448.csv ✔
../../data/app_processes/app_processes.query.5449.csv ✔
../../data/app_processes/app_processes.query.5450.csv ✔
../../data/app_processes/app_processes.query.5451.csv ✔
../../data/app_processes/app_processes.query.5452.csv ✔
../../data/app_processes/app_processes.query.5453.csv ✔
../../data/app_processes/app_processes.query.5454.csv ✔
../../data/app_processes/app_processes.query.5455.csv ✔
../../data/app_processes/app_processes.query.5456.csv ✔
../../data/app_processes/app_processes.query.5457.csv ✔
../../data/app_processes/app_processes.query.5458.csv ✔
../../data/app_processes/app_processes.query.5459.csv ✔
../../data/app_processes/app_processes.query.5460.csv ✔
../../data/app_processes/app_processes.query.5461.csv ✔
../../data/app_processes/app_processes.query.5462.csv ✔
../../data/app_processes/app_processes.query.546

../../data/app_processes/app_processes.query.5591.csv ✔
../../data/app_processes/app_processes.query.5592.csv ✔
../../data/app_processes/app_processes.query.5593.csv ✔
../../data/app_processes/app_processes.query.5594.csv ✔
Page 226 created!
../../data/app_processes/app_processes.query.5595.csv ✔
../../data/app_processes/app_processes.query.5596.csv ✔
../../data/app_processes/app_processes.query.5597.csv ✔
../../data/app_processes/app_processes.query.5598.csv ✔
../../data/app_processes/app_processes.query.5599.csv ✔
../../data/app_processes/app_processes.query.5600.csv ✔
../../data/app_processes/app_processes.query.5601.csv ✔
../../data/app_processes/app_processes.query.5602.csv ✔
../../data/app_processes/app_processes.query.5603.csv ✔
../../data/app_processes/app_processes.query.5604.csv ✔
../../data/app_processes/app_processes.query.5605.csv ✔
../../data/app_processes/app_processes.query.5606.csv ✔
../../data/app_processes/app_processes.query.5607.csv ✔
../../data/app_processes/app_p

../../data/app_processes/app_processes.query.5736.csv ✔
../../data/app_processes/app_processes.query.5737.csv ✔
../../data/app_processes/app_processes.query.5738.csv ✔
../../data/app_processes/app_processes.query.5739.csv ✔
../../data/app_processes/app_processes.query.5740.csv ✔
../../data/app_processes/app_processes.query.5741.csv ✔
../../data/app_processes/app_processes.query.5742.csv ✔
Page 232 created!
../../data/app_processes/app_processes.query.5743.csv ✔
../../data/app_processes/app_processes.query.5744.csv ✔
../../data/app_processes/app_processes.query.5745.csv ✔
../../data/app_processes/app_processes.query.5746.csv ✔
../../data/app_processes/app_processes.query.5747.csv ✔
../../data/app_processes/app_processes.query.5748.csv ✔
../../data/app_processes/app_processes.query.5749.csv ✔
../../data/app_processes/app_processes.query.5750.csv ✔
../../data/app_processes/app_processes.query.5751.csv ✔
../../data/app_processes/app_processes.query.5752.csv ✔
../../data/app_processes/app_p

../../data/app_processes/app_processes.query.5881.csv ✔
../../data/app_processes/app_processes.query.5882.csv ✔
../../data/app_processes/app_processes.query.5883.csv ✔
../../data/app_processes/app_processes.query.5884.csv ✔
../../data/app_processes/app_processes.query.5885.csv ✔
../../data/app_processes/app_processes.query.5886.csv ✔
../../data/app_processes/app_processes.query.5887.csv ✔
../../data/app_processes/app_processes.query.5888.csv ✔
../../data/app_processes/app_processes.query.5889.csv ✔
../../data/app_processes/app_processes.query.5890.csv ✔
../../data/app_processes/app_processes.query.5891.csv ✔
../../data/app_processes/app_processes.query.5892.csv ✔
../../data/app_processes/app_processes.query.5893.csv ✔
../../data/app_processes/app_processes.query.5894.csv ✔
Page 238 created!
../../data/app_processes/app_processes.query.5895.csv ✔
../../data/app_processes/app_processes.query.5896.csv ✔
../../data/app_processes/app_processes.query.5897.csv ✔
../../data/app_processes/app_p

In [15]:
if chunks:
    save_df_page(page, chunks, model_metadata)
    del chunks[:]
    del chunks
    gc.collect()

In [16]:
model_metadata.update_one({}, {'$set': {'total': total_rows}})

In [17]:
total_rows, total_memory_usage

(1396698198, 24480.990000000005)

In [18]:
content = '**{}** | task: data pre-processing, rows: {}, memory usage: {} MB' \
.format(model, total_rows, total_memory_usage)

requests.post(config['discord_webhook_url'], data={'content': content})

<Response [204]>